### Graph: a library for Havel Hakimi Tournaments

The `Graph` module offers data structures and methods for working with<br>
direct graphs in Haskell. The library is then extended to working examples<br>
of the Havel-Hakimi algorithm.

In [8]:
:l ./../Helpers -- mostly for qsort

data Vertex = V { name::String, degree::Int} deriving Eq
data Edge = E { source::Vertex, target::Vertex }
data Graph = G { edges::[Edge] } deriving Show
type Degrees = [Int]

vertices :: Graph -> [Vertex]
vertices = (map source).edges

degreesToVerts :: Degrees -> [Vertex]
degreesToVerts ds =  [V (show ss) d | (ss, d) <- zip [1..] ds]

instance Show Vertex where
  show (V a b) = a
instance Show Edge where
  show (E a b) = show a ++ "->" ++ show b

instance Ord Vertex where
  (<=) (V ss n) (V tt m) = n <= m
  (>=) (V ss n) (V tt m) = n >= m

The above data types `Vertex`, `Edge` and `Graph` are the heart of Graph module.<br>
Each comes with some default methods for accessing sub-types. Being defined explicitly,<br>
`vertices :: Graph -> [Vertex]` and `degreesToVerts :: Degrees -> [Vertex]`<br>
appear to be the odd methods out. `vertices` returns the vertices for a graph,<br>
while `degreesToVerts` allows us to build a vertex set with expectations for the degree<br>
of each vertex. Next, Some instances of `Show` are a included to keep things pretty.<br>
Lastly, `Ord` is extended to Vertex so that we can sort on them.<p>

Now the work horse functions:

In [12]:
vertsToGraph :: [Vertex] -> Graph
vertsToGraph verts = G $ hh [] verts
  where
    havel ((V ss n):as) = qsort $ snd_map (+ (-1)) (take n as) ++ drop n as
    toEdges ((V ss n):as) = [E (V ss n) vert | vert <- take n as]
    snd_map f xs =  [V a (f b)  | (V a b) <- xs]
    hh edgeAccum [] = edgeAccum
    hh edgeAccum verts =
      let sorted = qsort verts in
      hh (edgeAccum ++ toEdges sorted) (havel sorted)

degreesToGraph :: Degrees -> Graph
degreesToGraph = vertsToGraph.degreesToVerts

Now, given a list of expected degrees we can determine whether or not the list<br>
is Graphic in the sense of Erdős–Gallai. If it is then we produce such a graph<br>
via the Havel-Hakimi algorithm. For example a list of `n+1` n's ought to represent<br>
an `n-simplex` in our scheme.

In [14]:
simplex n = degreesToGraph.take (n+1) $ repeat n
simplex 4

G {edges = [5->4,5->3,5->2,5->1,4->3,4->2,4->1,3->2,3->1,2->1]}

We then see that a `4-simplex` is in fact graphic.